In [9]:
# extract list of firm names, cvm_codes, and b3 issuer from selected companies from selectef.csv
import pandas as pd
import numpy as np
import os
import sys
import re
import csv

# generate list 
def generate_list(file):
    df = pd.read_csv(file)
    headers = ['trade_name', 'cvm_code', 'b3_issuer', 'website','main_activity']
    df = df[df[headers]==headers]
    df = df.drop_duplicates(subset=['cvm_code'])
    df = df.dropna()
    return df

In [15]:

def generate_list(file):
    df = pd.read_csv(file)
    headers = ['trade_name', 'cvm_code', 'b3_issuer', 'website','main_activity']
    df = df[df[headers]==headers]
    df = df.drop_duplicates(subset=['name','cvm_code', 'b3_issuer'])  # Include 'b3_issuer' in the subset
    df = df.dropna()
    return df


,name,trade_name,cvm_code,about,b3_issuer_code,website,main_activity
0,Banco Bradesco S.A.,Banco Bradesco,906,"O Bradesco é um dos maiores bancos do Brasil, ...",BBDC,https://www.bradescori.com.br,Prática de Operações Bancárias em Geral. Inclu...
1,Banco Alfa de Investimento S.A.,Alfa Investimento,1384,NaN,BRIV,http://www.alfanet.com.br,Operações Permitidas à Sociedade Da Espécie em...
2,Companhia Siderúrgica Nacional,Siderúrgica Nacional,4030,A Companhia Siderúrgica Nacional (CSN) é uma d...,CSNA,https://ri.csn.com.br,A Companhia Siderúrgica Nacional Atua Nos Seto...
3,Braskem S.A.,Braskem,4820,A Braskem é uma empresa petroquímica global qu...,BRKM,https://www.braskem-ri.com.br,Produção de Produtos Químicos. Utilidades. Ene...
4,Lojas Americanas S.A.,Lojas Americanas,8087,A Americanas é uma empresa brasileira de varej...,LAME,http://ri.lasa.com.br,Comércio em Geral. Inclusive Supermercado E La...
5,Cosan S.A.,Cosan,19836,A Cosan S.A. é uma empresa brasileira com uma ...,CSAN,https://www.cosan.com.br,Produção de Açúcar. Etanol E Cogeração de Ener...
6,Vale S.A.,Vale,4170,A Vale S.A. é uma das maiores empresas de mine...,VALE,https://www.vale.com/pt/investidores,Mineração
7,Suzano Holding S.A.,Suzano Hold,9067,NaN,NEMO,http://www.suzano.com.br,Sociedade Holding - Participação Como Acionist...
8,Suzano S.A.,Suzano,13986,A Suzano Papel e Celulose é uma empresa brasil...,SUZB,https://ri.suzano.com.br,Indústria E Comércio de Papel E Celulose
9,Suzano S.A.,Suzano,13986,A Suzano Papel e Celulose é uma empresa brasil...,SUZB,https://ri.suzano.com.br,Indústria E Comércio de Papel E Celulose


In [29]:
import dontshareconfigCMS as dCMS
import requests
import logging
dados_de_mercado_api_key = dCMS.DDM_KEY

def fetch_news():
    logging.info("Fetching companies from Dados de Mercado API")
    url = "https://api.dadosdemercado.com.br/v1/news/"
    headers = {
    'Authorization': f'Bearer {dados_de_mercado_api_key}'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except Exception as e:
        logging.error(f"Error fetching companies: {e}")
        raise
    return response.json()

In [32]:
# get infos from fetch news json response
fetch_news_json = fetch_news()
fetch_news_json = pd.DataFrame(fetch_news_json)


In [33]:
fetch_news_json

,category,published_at,source,title,url
0,Economia,2024-06-18T09:15:00Z,Infomoney,Seguro para casa contra alagamento vai ficar m...,https://www.infomoney.com.br/minhas-financas/s...
1,Economia,2024-06-18T09:02:36Z,Bom dia Mercado,Fed boys falam hoje e Copom inicia reunião de ...,https://www.bomdiamercado.com.br/fed-boys-fala...
2,Legislação,2024-06-18T09:01:08.131000Z,Valor,Entenda o que é compliance e para que serve,https://valor.globo.com/empresas/noticia/2024/...
3,Economia,2024-06-18T09:00:48Z,Bom dia Mercado,Vai rolar: Fed boys falam e Copom inicia reuni...,https://www.bomdiamercado.com.br/vai-rolar-fed...
4,Brasil,2024-06-18T09:00:41.218000Z,Valor,"Saiba onde 24 de junho, dia de São João, é fer...",https://valor.globo.com/brasil/noticia/2024/06...
...,...,...,...,...,...
95,Mercados,2024-06-17T19:52:47.707000Z,Valor,Bilionário Ken Fisher mais que dobrará fortuna...,https://valor.globo.com/financas/noticia/2024/...
96,Política,2024-06-17T19:51:34Z,Infomoney,Lula critica burocracia ao assinar contrato do...,https://www.infomoney.com.br/politica/lula-cri...
97,Brasil,2024-06-17T19:50:21.980000Z,Valor,Brasil ganha três Leões de Ouro na abertura do...,https://valor.globo.com/empresas/marketing/can...
98,Política,2024-06-17T19:44:18.921000Z,Valor,"Cotado para 2026, Tarcísio diz não ter interes...",https://valor.globo.com/politica/noticia/2024/...
